# Collection, Parsing, and Processing of Stock Data for Four Stocks

In [57]:
from pathlib import Path
import urllib as ul
import bs4
import datetime
import pandas as pd
import json

Stocks used: Amazon, Netflix, Delta Air Lines, Ford 

Declaring important data:

In [58]:
# prefix for fetching the data
html_prefix = "http://mlg.ucd.ie/modules/COMP30760/stocks/data-"

# stocks for study
stock_names = ["Amazon", "Netflix", "Delta Air Lines", "Ford"]

# codes which the stock market uses to label these companies
stock_ids = {"Amazon":"AMZN", "Netflix":"NFLX", "Delta Air Lines":"DAL", "Ford":"F"}

# creating a data directory in case it doesnt already exist
dir_data = Path("data")
dir_data.mkdir(parents=True, exist_ok=True)

# Data Collection

Function which retrieves the data from the html page of any given stock

Example link to recreate: http://mlg.ucd.ie/modules/COMP30760/stocks/data-amzn.html

In [59]:
def data_collect(stock_name):
    # make sure the stock code is in lower case for the link to work
    link = html_prefix + stock_ids[stock_name].lower() + ".html"
    response = ul.request.urlopen(link)
    html = response.read().decode()
    return html

Function which finds the order in which the columns are stored on the html page

In [60]:
def find_cols(html):
    parser = bs4.BeautifulSoup(html,"html.parser")
    columnshtml = parser.find("thead")
    attributes = []
    for match in columnshtml.find_all("td"):
        text = match.get_text()
        text = text.strip()
        attributes.append(text)
    return attributes

Converts the raw data to a dataframe indexed by date

In [61]:
def parse_raw_data(stock_name, html):
    cols = find_cols(html)
    rows = []
    parser = bs4.BeautifulSoup(html,"html.parser")
    # finds each table in the data set
    for match in parser.find_all("tbody"):
        # finds each row in the table
        for data_row in match.find_all("tr"):
            items = []
            
            # cleans the data in the row so that it is convenient to use
            for value in data_row.find_all("td"):
                item = value.get_text()
                item = item.strip()
                items.append(item)

            # if the first columns item does not mathc the stock code then the row is blank and it is ignored
            if items[0] == stock_ids[stock_name]:
                row = {}
                
                # get index of each of the columns relating tp the date
                daycol = cols.index("Day")
                monthcol = cols.index("Month")
                yearcol = cols.index("Year")
                
                #convert the month to a number no matter what way it is stored
                if items[monthcol].isnumeric():
                    month_holder = datetime.datetime.strptime(items[monthcol], "%m")
                else:
                    #slice so that months stored as full words also can be treated as shortened months
                    month_holder = datetime.datetime.strptime(items[monthcol][:3], "%b")
                month_num = month_holder.month
                
                row["Date"] = datetime.datetime(int(items[cols.index("Year")]), month_num, int(items[cols.index("Day")]))

                # close is the only necssary data to record
                row["Close"] = float(items[cols.index("Close")])

                rows.append(row)
    
    df = pd.DataFrame(rows)
    return df

Creating a dataframe of each stock and recording them in a dictionary

In [62]:
df_stocks = {}
for stock_name in stock_names:
    html = data_collect(stock_name)
    df_stocks[stock_name] = parse_raw_data(stock_name, html)
    df_stocks[stock_name] = df_stocks[stock_name].set_index("Date")

## Data Verification

Verify that the stats were recorded for each stock

In [63]:
for stock_name in stock_names:
    print(df_stocks[stock_name].head(5))

                  Close
Date                   
2019-10-01  1735.650024
2019-10-02  1713.229980
2019-10-03  1724.420044
2019-10-04  1739.650024
2019-10-07  1732.660034
                 Close
Date                  
2019-10-01  269.579987
2019-10-02  268.029999
2019-10-03  268.149994
2019-10-04  272.790009
2019-10-07  274.459991
                Close
Date                 
2019-10-01  57.009998
2019-10-02  54.349998
2019-10-03  52.830002
2019-10-04  53.810001
2019-10-07  53.360001
            Close
Date             
2019-10-01   8.90
2019-10-02   8.61
2019-10-03   8.71
2019-10-04   8.74
2019-10-07   8.68


Check for any null values in the stocks

In [64]:
for stock_name in stock_names:
    print(df_stocks[stock_name].isnull().sum())

Close    26
dtype: int64
Close    29
dtype: int64
Close    24
dtype: int64
Close    38
dtype: int64


Replacing all the null values with the previous days values

In [65]:
for stock_name in stock_names:
    if(df_stocks[stock_name].isnull().sum().sum() != 0):
        df_stocks[stock_name] = df_stocks[stock_name].fillna(method = 'ffill')
        
    # if the first values in the dataframes are null it will replace them with the next value instead of the previous which doesnt exist
    if(df_stocks[stock_name].isnull().sum().sum() != 0):
        df_stocks[stock_name] = df_stocks[stock_name].fillna(method = 'bfill')
        
    print(df_stocks[stock_name].isnull().sum())

Close    0
dtype: int64
Close    0
dtype: int64
Close    0
dtype: int64
Close    0
dtype: int64


Writing the data to a file 

In [66]:
def store(stock_name): 
    fname = "%s.json" % stock_ids[stock_name]
    out_path = dir_data / fname
    print("Writing %s" % out_path)
    df_stocks[stock_name].to_json(out_path, orient="index")

In [67]:
for stock_name in stock_names:
    store(stock_name)

Writing data\AMZN.json
Writing data\NFLX.json
Writing data\DAL.json
Writing data\F.json
